# Fix gene ids in iYali map

This notebook fixes the iYali gene ids available at https://caffeine.dd-decaf.eu/

Benjamín J. Sánchez, 2020-03-23

## 1. Getting model and map

* Model: https://api.dd-decaf.eu/model-storage/models/44
* Map: https://api.dd-decaf.eu/map-storage/maps/37

In [1]:
import wget
wget.download("https://api.dd-decaf.eu/model-storage/models/44", "iYali-model.json", bar=False)
wget.download("https://api.dd-decaf.eu/map-storage/maps/37", "iYali-map.json", bar=False)

'iYali-map.json'

Let's check if they are the same in staging:

* Model: https://api-staging.dd-decaf.eu/model-storage/models/44
* Map: https://api-staging.dd-decaf.eu/map-storage/maps/90

In [2]:
import os
os.remove("iYali-model.json")
os.remove("iYali-map.json")
wget.download("https://api-staging.dd-decaf.eu/model-storage/models/44", "iYali-model.json", bar=False)
wget.download("https://api-staging.dd-decaf.eu/map-storage/maps/90", "iYali-map.json", bar=False)  # note the id is different

'iYali-map.json'

So the model is the same (no changes in git) but the map is different.